In [ ]:
// 1. import axios
import axios from "axios";

// 2. Cấu hình instance chứa các config
const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
});

axios.interceptors.response.use((response) => {
    return response.data;
});

let isRefreshing = false;
let failedQueue = [];

/* Thêm Request lỗi vào hàng chờ */
const addToQueue = (resolve, reject) => {
    failedQueue.push({ resolve, reject });
};

/* Xử lý hàng đợi sau khi Refresh Token xong! */
const processQueue = (error) => {
    // Lặp qua từng Promise của các request lỗi
    failedQueue.forEach((prom) => {
        if (error) {
            prom.reject(error);
        } else {
            // Resolve để báo hiệu rằng "Đã refresh xong"
            prom.resolve();
        }
    });

    // Xóa hàng đợi
    failedQueue = [];
};

/* Hàm gọi Refresh Token */
const refreshAccessToken = async (baseURL) => {
    try {
        // Lấy refreshToken từ localStorage
        const refreshToken = localStorage.getItem("refreshToken");

        // Gọi API Refresh với Axios
        const response = await axios.post(`${baseURL}/auth/refresh-token`, {
            refresh_token: refreshToken,
        });

        // Lấy được cặp token mới
        // Lưu vào localStorage
        const { access_token, refresh_token } = response.data;
        localStorage.setItem("accessToken", access_token);
        localStorage.setItem("refreshToken", refresh_token);

        // Trả ra access_token mới
        return access_token;
    } catch (error) {
        // Nếu refresh Lỗi
        // Xoá sạch
        // Throw lỗi ra bên ngoài
        localStorage.removeItem("accessToken");
        localStorage.removeItem("refreshToken");
        throw error;
    }
};

/* ==========================================================
 * httpRequest interceptors
 * ==========================================================*/

/* Request */
httpRequest.interceptors.request.use(
    (config) => {
        const accessToken = localStorage.getItem("accessToken");
        if (accessToken) config.headers.Authorization = `Bearer ${accessToken}`;

        return config;
    },
    (error) => Promise.reject(error),
);

/* Response */
httpRequest.interceptors.response.use(
    (response) => {
        return response.data;
    },
    async (error) => {
        // 0. Lấy refreshToken & cục config gốc
        const originalRequest = error.config;

        /* 
        ❌ Lỗi không phải 401
        ❌ Request này đã thử lại rồi mà vẫn Lỗi
            =>> Reject
        */
        if (error.status !== 401 || originalRequest._retry) {
            return Promise.reject(error);
        }

        // 1. Đánh dấu đã retry (thử lại)
        originalRequest._retry = true;

        /* Case 1: Đang trong quá trình refreshing */
        if (isRefreshing) {
            // Các request đến sau phải chờ & được xếp vào hàng đợi
            return new Promise((resolve, reject) => {
                addToQueue(resolve, reject);
            })
                .then(() => {
                    // Khi được resolve, gọi lại request cũ
                    return httpRequest(originalRequest);
                })
                .catch((error) => Promise.reject(error));
        }

        /* Case 2: Request đầu tiên gặp lỗi */
        // 2. Đánh dấu là đang refresh
        isRefreshing = true;

        try {
            // 3. Gọi API lấy token mới
            // Lưu vào localStorage
            await refreshAccessToken(originalRequest.baseURL);

            // 4. Refresh thành công
            // Báo cho các request đang chờ trong hàng đợi 🏃‍♂️‍➡️🏃‍♂️‍➡️🏃‍♂️‍➡️chạy lại
            processQueue(null);

            // 5. Gọi lại request lỗi của chính mình
            return httpRequest(originalRequest);
        } catch (error) {
            // Refresh thất bại => Báo lỗi cho toàn bộ hàng đợi
            processQueue(error);

            // Xử lý logout hoặc đá về trang login (nếu cần)
            // window.location.href = '/login';

            return Promise.reject(error);
        } finally {
            // Luôn tắt cờ refresh dù thành công hay thất bại
            isRefreshing = false;
        }
    },
);

export default httpRequest;
